# Limpieza del resto de columnas

### Ahora que ya tenemos las skills tecnológicas identificadas, procederemos a tratar el resto de columnas que sean necesarias para posterirmente, generar nuestros jsons e introduccirlos en nuestra base de datos.

---
---

## Importamos Librerias y Configuraciones

In [10]:
import os
import pandas as pd
import re
from tqdm import tqdm
from IPython.core.display import display, HTML
from typing import List

C:\Users\JLFA\AppData\Local\Temp\ipykernel_21308\847137472.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [11]:
# generamos nuestro diccionario con los dataframes de nuestras ofertas laborales

ruta = '../datos/datos_recibidos_empresa/datos_skills_identificadas/'
dic_dfs_ofertas = {archivo.rstrip('.csv') : pd.read_csv(f'{ruta}{archivo}') for archivo in os.listdir(ruta) if archivo.endswith('.csv')}

dic_dfs_ofertas.keys()

dict_keys(['ofertas_data_analyst_identificada', 'ofertas_data_engineer_identificada', 'ofertas_data_scientist_identificada', 'ofertas_seguridad_privacy_data_identificada'])

## Detectamos elementos que queramos corregir de nuestros datos
#### vamos a coger un dataframe de ejemplo ya que todos ellos tienen las mismas columnas

In [12]:
df_prueba = dic_dfs_ofertas['ofertas_data_engineer_identificada']

df_prueba.sample(3)

,Titulo,Empresa,Ubicacion,Salario,Experiencia minima,Tipo de contrato,Estudios minimos,Skills,Descripcion,Referencia,cat_original,cat_identificada
298,Cloud Data Engineer,EOS Solutions,"Málaga, España",35.000€ - 45.000€ brutos anuales,Al menos 5 años,"Indefinido, jornada completa",Curso especializado en Big Data,"['apache kafka', 'airflow', 'nosql']",**Cloud Data Engineer en EOS Solutions**\n\n ...,DENG18296,Data Engineer,NaN
267,Data Architect,Gi Group,"Madrid, España",Salario no disponible,Sin experiencia,"Indefinido, jornada parcial","Grado en Informática, Matemáticas o similar","['python', 'sql', 'apachespark', 'hadoop', 'aws']",**Data Architect en Gi Group**\n\n ¿Estás l...,BDENG91064,Data Engineer,NaN
206,Cloud Data Engineer,BETWEEN,"Alicante, España",35.000€ - 45.000€ brutos anuales,Sin experiencia,"Contrato por proyecto, freelance",FP en Desarrollo de Aplicaciones Multiplataforma,"['apache kafka', 'airflow', 'nosql']",**Cloud Data Engineer en BETWEEN**\n\n ¿Est...,BDENG91165,Data Engineer,NaN


#### Elementos detectados para tratar:
<ul>
    <li><strong>Salario</strong></li>
        <ul>
            <li>Eliminar €</li>
            <li>Quedarnos con solos los numeros</li>
            <li>Crear lista con la banda salarial o un unico valor en caso de ser solo un numero</li>
        </ul>
    <li><strong>Estudios minimos</strong></li>
        <ul>
            <li>Separar por ',' y crear una lista</li>
        </ul>
    <li><strong>Descripcion</strong></li>
        <ul>
            <li>Eliminar el caracter '*'</li>
        </ul>
</ul>

(hay otros tratamientos como los años de experiencia y demás que los dejaremos como 'Next Steps' debido a que ahora no los utilizaremos para filtrar ni nada, son solo datos informativos)

#### Tratamiento columna `Salario`

In [13]:
# # utilizamos regex para quedarnos solo con los numeros y mediante un ternario, creamos una lista con el valor 'No Disponible' en caso de que proceda
# df_prueba['Salario'] = df_prueba['Salario'].map(lambda x : re.findall(r'\d+(?:\.\d+)?', x) if x != 'Salario no disponible' else ['No Dispobible'])

# display(HTML("<h2 style='color: orange'>Así quedaría ahora nuestra columna <u>Salario</u></h2>"))
# df_prueba.sample(3)

#### Tratamiento columna `Estudios minimos`

In [14]:
# df_prueba['Estudios minimos'] = df_prueba['Estudios minimos'].map(lambda x : x.split(',') if x != None else ['Desconocidos'])

# display(HTML("<h2 style='color: orange'>Así quedaría ahora nuestra columna <u>Estudios minimos</u></h2>"))
# df_prueba.sample(3)

#### Tratamiento columna `Descripcion`

In [15]:
# df_prueba['Descripcion'] = df_prueba['Descripcion'].map(lambda x : x.replace('*','') if x != None else None)

# display(HTML("<h2 style='color: orange'>Así quedaría ahora nuestra columna <u>Descripcion</u></h2>"))
# df_prueba.sample(3)

## Realizamos proceso de limpieza

#### Ahora que hemos visto que cubrimos los casos que nos interesaban de forma adecuada, creamos la funcion de limpieza

In [16]:


def limpiar_formato_dfs(dict_dfs: List[pd.DataFrame]):

    resultado = {}
    for key, value in tqdm(dict_dfs.copy().items(), desc="Limpiando Dfs: "):

        value['Salario'] = value['Salario'].map(lambda x : re.findall(r'\d+(?:\.\d+)?', x) if isinstance(x, str) and x != 'Salario no disponible' else ['No Dispobible'])
        value['Estudios minimos'] = value['Estudios minimos'].map(lambda x : x.split(',') if x != None else ['Desconocidos'])
        value['Descripcion'] = value['Descripcion'].map(lambda x : x.replace('*','') if x != None else None)

        resultado[key] = value

    return resultado

In [17]:
dfs_limpios = limpiar_formato_dfs(dic_dfs_ofertas)

Limpiando Dfs: 100%|██████████| 4/4 [00:00<00:00, 269.66it/s]


## Guardamos los DFs procesados

In [18]:
for key, value in dfs_limpios.items():
    nombre = key.replace('_identificada','_tratadas')
    value.to_csv(f'../datos/datos_recibidos_empresa/datos_skills_identificadas/tratados/{nombre}.csv', index=False)